In [1]:
import sqlite3

In [2]:
import yaml

In [3]:
from pydantic import BaseModel
from typing import List
from enum import Enum
from pathlib import Path
import re
from wcmatch.glob import globmatch
from typing import Tuple, Optional

# PermissionRule

In [4]:
# util
def issubpath(path1, path2):
    return path1 in path2.parents

In [5]:
from pydantic import model_validator


class PermissionType(Enum):
    CREATE=1
    READ=2
    WRITE=3
    ADMIN=4
    
    
class PermissionParsingError(Exception):
    pass

class PermissionRule(BaseModel):
    dir_path: Path #where does this permfile live
    path: str # what paths does it apply to (e.g. **/*.txt)
    user: str # can be *, 
    allow: bool = True
    terminal: bool = False
    permissions: List[PermissionType] # read/write/create/admin
    priority: int
    
    def __eq__(self, other):
        return self.model_dump() == other.model_dump()
    
    @property
    def permfile_path(self):
        return self.dir_path / '.syftperm'
    
        # write model validator that accepts either a single string or a list of strings as permissions when initializing
    @model_validator(mode='before')
    @classmethod
    def validate_permissions(cls, values):
        # check if values only contains keys that are in the model        
        invalid_keys = set(values.keys()) - (set(cls.model_fields.keys()) | set(["type"]))
        if len(invalid_keys) > 0:
            raise PermissionParsingError(f"rule yaml contains invalid keys {invalid_keys}, only {cls.model_fields.keys()} are allowed")
        
        # add that if the type value is "disallow" we set allow to false
        if values.get('type') == "disallow":
            values['allow'] = False
            
        # if path refers to a location higher in the directory tree than the current file, raise an error
        if values.get('path').startswith('../'):
            raise PermissionParsingError(f"path {values.get('path')} refers to a location higher in the directory tree than the current file")
        
        # if user is not a valid email, or *, raise an error
        email = values.get('user')
        if email != "*" and not bool(re.search(r"^[\w\.\+\-]+\@[\w]+\.[a-z]{2,3}$", email)):
            raise PermissionParsingError(f"user {values.get('user')} is not a valid email or *")
        
        # listify permissions
        perms = values.get('permissions')
        if isinstance(perms, str):
            perms = [perms]
        if isinstance(perms, list):
            values['permissions'] = [
                PermissionType[p.upper()] if isinstance(p, str) else p 
                for p in perms
            ]
        else:
            raise ValueError(f"permissions should be a list of strings or a single string, received {type(perms)}")
        
        path = values.get('path')
        if "**" in path and "{useremail}" in path and path.index("**") < path.rindex("{useremail}"):
            # this would make creating the path2rule mapping more challenging to compute beforehand
            raise PermissionParsingError("** can never be after {useremail}")
        
        return values
    
    @classmethod
    def from_rule_dict(cls, dir_path, rule_dict, priority):
        # initialize from dict
        return cls(dir_path=dir_path, **rule_dict, priority=priority)
    
    @classmethod
    def from_db_row(cls, row):
        """Create a PermissionRule from a database row"""
        permissions = []
        if row['can_read']:
            permissions.append(PermissionType.READ)
        if row['can_create']:
            permissions.append(PermissionType.CREATE) 
        if row['can_write']:
            permissions.append(PermissionType.WRITE)
        if row['admin']:
            permissions.append(PermissionType.ADMIN)
            
        return cls(
            dir_path=Path(row['permfile_path']).parent,
            path=row['path'],
            user=row["user"],  # Default to all users since DB schema doesn't show user field
            allow=not row['disallow'],
            terminal=bool(row['terminal']),
            priority=row['priority'],
            permissions=permissions
        )
    
    def to_db_row(self):
        """Convert PermissionRule to a database row dictionary"""
        return {
            'permfile_path': str(self.permfile_path),  # Reconstruct full path
            'permfile_dir': str(self.dir_path),
            "priority": self.priority,
            'path': self.path,
            'user': self.user,
            'can_read': PermissionType.READ in self.permissions,
            'can_create': PermissionType.CREATE in self.permissions,
            'can_write': PermissionType.WRITE in self.permissions,
            'admin': PermissionType.ADMIN in self.permissions,
            'disallow': not self.allow,
            'terminal': self.terminal
        }
    
    @property
    def permission_dict(self):
        return {
            "read": PermissionType.READ in self.permissions,
            "create": PermissionType.CREATE in self.permissions,
            "write": PermissionType.WRITE in self.permissions,
            "admin": PermissionType.ADMIN in self.permissions,
        }

    def filepath_matches_rule_path(self, filepath: Path) -> Tuple[bool, Optional[str]]:
        if issubpath(self.dir_path, filepath):
            relative_file_path = filepath.relative_to(self.dir_path)
        else:
            return False, None
        
        match_for_email = None
        if self.has_email_template:
            match = False
            emails_in_file_path = [part for part in relative_file_path.split("/") if "@" in part] # todo: improve this
            for email in emails_in_file_path:
                if globmatch(self.path.replace("{useremail}", email), str(relative_file_path)):
                    match=True
                    match_for_email = email
                    break
        else:
            match = globmatch(self.path, str(relative_file_path))
        return match, match_for_email

    @property
    def has_email_template(self):
        return "{useremail}" in self.path

    def resolve_path_pattern(self, email):
        return self.path.replace("{useremail}", email)

# PermissionFile

In [6]:
class PermissionFile(BaseModel):
    filepath: Path
    content: str
    rules: List[PermissionRule]
    
    @property
    def dir_path(self):
        return self.filepath.parent

    @classmethod
    def from_file(cls, path):
        with open(path, "r") as f:
            rule_dicts = yaml.safe_load(f)
            content = f.read()
            return cls.from_rule_dicts(path, content, rule_dicts)

    @classmethod
    def from_rule_dicts(cls, permfile_file_path, content, rule_dicts):
        if not isinstance(rule_dicts, list):
            raise ValueError(f"rules should be passed as a list of dicts, received {type(rule_dicts)}")
        rules = []
        dir_path = Path(permfile_file_path).parent
        for i, rule_dict in enumerate(rule_dicts):
            rule = PermissionRule.from_rule_dict(dir_path, rule_dict, priority=i)
            rules.append(rule)
        return cls(filepath=permfile_file_path, content=content, rules=rules)
    
    @classmethod
    def from_string(cls, s, path):
        dicts = yaml.safe_load(s)
        return cls.from_rule_dicts(Path(path), s, dicts)
    
    
    

## Parsing tests

In [7]:
yaml_string = """
- permissions: read
  path: x.txt
  user: user@example.org
  
- permissions: [read, write]
  path: x.txt
  user: "*"
  type: disallow
  terminal: true
"""

In [8]:
rule = PermissionRule.from_rule_dict(Path("."), yaml.safe_load(yaml_string)[0], priority=0)

In [9]:
yaml.safe_load(yaml_string)

[{'permissions': 'read', 'path': 'x.txt', 'user': 'user@example.org'},
 {'permissions': ['read', 'write'],
  'path': 'x.txt',
  'user': '*',
  'type': 'disallow',
  'terminal': True}]

In [10]:
file = PermissionFile.from_string(yaml_string, ".")

In [11]:
assert len(file.rules) == 2

assert file.rules[0].permissions == [PermissionType.READ]
assert file.rules[0].path == "x.txt"
assert file.rules[0].user == "user@example.org"
assert file.rules[0].allow == True
assert file.rules[0].terminal == False


#check the same for the second rule
assert file.rules[1].permissions == [PermissionType.READ, PermissionType.WRITE]
assert file.rules[1].path == "x.txt"
assert file.rules[1].user == "*"
assert file.rules[1].allow == False
assert file.rules[1].terminal == True



In [12]:
import pytest

In [13]:
yaml_string = """
- permissions: read
  path: "../*/x.txt"
  user: user@example.org
"""
with pytest.raises(PermissionParsingError):
    PermissionFile.from_string(yaml_string, ".")

In [14]:
yaml_string = """
- permissions: read
  path: "{useremail}/*"
  user: user@example.org
"""
file = PermissionFile.from_string(yaml_string, ".")
rule = file.rules[0]
assert rule.has_email_template
assert rule.resolve_path_pattern("user@example.org") == "user@example.org/*"

In [15]:
yaml_string = """
- permissions: read
  path: "{useremail}/*"
  userx: user@example.org
"""

In [16]:
from tempfile import NamedTemporaryFile

yaml_string = """
- permissions: read
  path: "{useremail}/*"
  user: user@example.org
"""

with NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as tmp:
    tmp.write(yaml_string)
    tmp.flush()
    file = PermissionFile.from_file(tmp.name)


# Tables

In [17]:
import sqlite3
connection = sqlite3.connect(":memory:")
connection.row_factory = sqlite3.Row  # Set row factory to sqlite3.Row
# connection.row_factory = sqlite3.Row  # Set row factory to sqlite3.Row


# Create a cursor object
cursor = connection.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

# Create a table for storing file information
res = cursor.execute("""
    CREATE TABLE rules (
        permfile_path varchar(1000) NOT NULL,
        permfile_dir varchar(1000) NOT NULL,
        priority INTEGER NOT NULL,
        path varchar(1000) NOT NULL,
        user varchar(1000) NOT NULL,
        can_read bool NOT NULL,
        can_create bool NOT NULL,
        can_write bool NOT NULL,
        admin bool NOT NULL,
        disallow bool NOT NULL,
        terminal bool not null,
        PRIMARY KEY (permfile_path, priority)
    )
""")

In [18]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS file_metadata (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    path TEXT NOT NULL UNIQUE,
    hash TEXT NOT NULL,
    signature TEXT NOT NULL,
    file_size INTEGER NOT NULL,
    last_modified TEXT NOT NULL        
)
""")

In [19]:

cursor.execute("""
CREATE TABLE rule_files (
    permfile_path varchar(1000) NOT NULL,
    priority INTEGER NOT NULL,
    file_id INTEGER NOT NULL,
    match_for_email varchar(1000),
    PRIMARY KEY (permfile_path, priority, file_id),
    FOREIGN KEY (permfile_path, priority) REFERENCES rules(permfile_path, priority) ON DELETE CASCADE,
    FOREIGN KEY (file_id) REFERENCES file_metadata(id) ON DELETE CASCADE
);
""")


In [20]:
connection.commit()

# Define queries and inserts

In [21]:
def query_rules_for_permfile(cursor, file: PermissionFile):
    cursor.execute("""
        SELECT * FROM rules WHERE permfile_path = ? ORDER BY priority
    """, (str(file.filepath),))
    return cursor.fetchall()    

def get_rules_for_permfile(cursor, file: PermissionFile):
    return [PermissionRule.from_db_row(row) for row in query_rules_for_permfile(cursor, file)]

In [22]:
def get_permission_table_column_names(cursor):
    return [dict(x)["name"] for x in cursor.execute("""
        SELECT name, type FROM pragma_table_info('rules')
    """).fetchall()]


In [23]:
def get_all_files_under_dir(cursor, dir_path):
    cursor.execute("""
        SELECT * FROM file_metadata WHERE path LIKE ?
    """, (str(dir_path) + "/%",))
    return cursor.fetchall()


In [24]:
from syftbox.server.sync.models import FileMetadata


def get_all_files(cursor):
    cursor.execute("""
        SELECT * FROM file_metadata
    """)
    return cursor.fetchall()

def get_all_files_under_syftperm(cursor, permfile: PermissionFile) -> List[Path]:
    cursor.execute("""
        SELECT * FROM file_metadata WHERE path LIKE ?
    """, (str(permfile.dir_path) + "/%",))
    return [(row["id"], FileMetadata(path=Path(row["path"]), hash=row["hash"], signature=row["signature"], file_size=row["file_size"], last_modified=row["last_modified"]))
            for row in cursor.fetchall()]



In [25]:
def insert_file_mock(path):
    cursor.execute("""
        INSERT INTO file_metadata (path, hash, signature, file_size, last_modified) VALUES (?, ?, ?, ?, ?)
    """, (path, "dummy_hash", "dummy_signature", 1234, "2023-01-01"))
    connection.commit()
    return cursor.lastrowid

    

In [26]:
def get_all_file_mappings(cursor):
    cursor.execute("""
        SELECT permfile_path, priority, file_id, match_for_email 
        FROM rule_file_mappings
    """)
    rows = cursor.fetchall()
    return [
        {
            "permfile_path": row[0],
            "priority": row[1], 
            "file_id": row[2],
            "match_for_email": row[3]
        }
        for row in rows
    ]


In [27]:
def set_rules_for_permfile(connection, file: PermissionFile):
    """
    Atomically set the rules for a permission file. Basically its just a write operation, but
    we also make sure we delete the rules that are no longer in the file.
    """
    try:
        cursor = connection.cursor()
        
        cursor.execute(
        """
        DELETE FROM rules 
        WHERE permfile_path = ? 
        """,  
            (str(file.filepath),)
        )

            
        # TODO
        files_under_dir = get_all_files_under_syftperm(cursor, file)
        
        rule2files = []
        
        for rule in file.rules:
            for _id, file_in_dir in files_under_dir:
                match, match_for_email = rule.filepath_matches_rule_path(file_in_dir.path)
                if match:
                    rule2files.append([str(rule.permfile_path), rule.priority, _id, match_for_email])                
        
        rule_rows = [tuple(rule.to_db_row().values()) for rule in file.rules]
            
        cursor.executemany("""
        INSERT INTO rules (
            permfile_path, permfile_dir, priority, path, user, 
            can_read, can_create, can_write, admin, 
            disallow, terminal
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ON CONFLICT(permfile_path, priority) DO UPDATE SET
            path = excluded.path,
            user = excluded.user,
            can_read = excluded.can_read,
            can_create = excluded.can_create,
            can_write = excluded.can_write,
            admin = excluded.admin,
            disallow = excluded.disallow,
            terminal = excluded.terminal;
        """, rule_rows)
        
        
        cursor.executemany("""
            INSERT INTO rule_files (permfile_path, priority, file_id, match_for_email) VALUES (?, ?, ?, ?)
        """, rule2files)
        
        
        connection.commit()
    except Exception as e:
        connection.rollback()
        raise e



# TODO

# Test

## Test adding rules and their rule_file_mappings (`rule_files`)

In [28]:
if len(get_all_files(cursor)) == 0:
    for f in ["a.txt", "b.txt", "c.txt"]:
        insert_file_mock(f"user@example.org/test2/{f}")


yaml_string = """
- permissions: read
  path: a.txt
  user: user@example.org
  
- permissions: write
  path: b.txt
  user: user@example.org
  
- permissions: write
  path: z.txt
  user: "*"
  type: disallow
  terminal: true
"""
file_path = "user@example.org/test2/.syftperm"
file = PermissionFile.from_string(yaml_string, file_path)


set_rules_for_permfile(connection, file)

assert len(get_all_file_mappings(cursor)) == 2




OperationalError: no such table: rule_file_mappings

## Test overwriting rules

In [29]:
yaml_string = """
- permissions: read
  path: a.txt
  user: user@example.org
  
- permissions: write
  path: b.txt
  user: user@example.org
  
- permissions: write
  path: z.txt
  user: "*"
  type: disallow
  terminal: true
"""
file_path = "user@example.org/test2/.syftperm"
file = PermissionFile.from_string(yaml_string, file_path)
set_rules_for_permfile(connection, file)
written_rules = get_rules_for_permfile(cursor, file)

permissions = [x.permissions for x in written_rules]
users = [x.user for x in written_rules]
terminals = [x.terminal for x in written_rules]
allows = [x.allow for x in written_rules]
assert len(written_rules) == 3
assert permissions == [[PermissionType.READ], [PermissionType.WRITE], [PermissionType.WRITE]]
assert users == ["user@example.org", "user@example.org", "*"]
assert terminals == [False, False, True]
assert allows == [True, True, False]


assert len([x for x in get_all_file_mappings(cursor) if x["permfile_path"] == str(file.filepath)]) == 2



In [30]:
# " | ".join(get_permission_table_column_names(cursor))   


In [31]:
yaml_string = """
- permissions: read
  path: a.txt
  user: user@example.org
  
- permissions: create
  path: x.txt
  user: user@example.org
  
- permissions: create
  path: z.txt
  user: "*"
  type: disallow
  terminal: true
  
- permissions: create
  path: d.txt
  user: "*"
  terminal: true
"""

file_path = "user@example.org/test2/.syftperm"
file = PermissionFile.from_string(yaml_string, file_path)
set_rules_for_permfile(connection, file)

In [32]:
new_existing_rules = get_rules_for_permfile(cursor, file)
paths = [x.path for x in new_existing_rules]
permissions = [x.permissions for x in new_existing_rules]
users = [x.user for x in new_existing_rules]
terminals = [x.terminal for x in new_existing_rules]
allows = [x.allow for x in new_existing_rules]
assert len(new_existing_rules) == 4
assert paths == ["a.txt", "x.txt", "z.txt", "d.txt"]
assert permissions == [[PermissionType.READ], [PermissionType.CREATE], [PermissionType.CREATE], [PermissionType.CREATE]]
assert users == ["user@example.org", "user@example.org", "*", "*"]
assert terminals == [False, False, True, True]
assert allows == [True, True, False, True]

In [33]:
assert len(get_all_file_mappings(cursor)) == 1

## Get single permission for a user

In [34]:
def get_rules_for_path(cursor, path: Path):
    parents = path.parents
    placeholders = ",".join("?" * len(parents))
    cursor.execute("""
        SELECT * FROM rules WHERE permfile_dir in ({})
    """.format(placeholders), [str(x) for x in parents])
    return [PermissionRule.from_db_row(row) for row in cursor.fetchall()]


In [35]:
class ComputedPermission(BaseModel):
    user: str
    file_path: Path
    terminal: dict[PermissionType, bool] = {
        PermissionType.READ: False,
        PermissionType.CREATE: False,
        PermissionType.WRITE: False,
        PermissionType.ADMIN: False,
    }
    
    perms: dict[PermissionType, bool] = {
        PermissionType.READ: False,
        PermissionType.CREATE: False,
        PermissionType.WRITE: False,
        PermissionType.ADMIN: False,
    }
    
    
    @classmethod
    def from_user_and_path(cls, cursor: sqlite3.Cursor, user: str, path: Path):
        rules: List[PermissionRule] = get_rules_for_path(cursor, path)
    
        permission = cls(user=user, file_path=path)                           
        for rule in rules:
            permission.apply(rule)
            
        return permission

    def has_permission(self, permtype: PermissionType):
        return self.perms[permtype]
        
    def user_matches(self, rule: PermissionRule):
        """Computes if the user in the rule"""
        if rule.user == "*":
            return True
        elif rule.user == self.user:
            return True
        else:
            return False

    def rule_applies_to_path(self, rule: PermissionRule):
        if rule.has_email_template:
            # we fill in a/b/{useremail}/*.txt -> a/b/user@email.org/*.txt
            resolved_path_pattern = rule.resolve_path_pattern(self.user)
        else:
            resolved_path_pattern = rule.path
            
        # target file path (the one that we want to check permissions for relative to the syftperm file
        # we need this because the syftperm file specifies path patterns relative to its own location
        
        if issubpath(rule.dir_path, self.file_path):
            relative_file_path = self.file_path.relative_to(rule.dir_path)
            return globmatch(relative_file_path, resolved_path_pattern)
        else:
            return False
            
    def apply(self, rule: PermissionRule):
        # TODO: is terminal on a rule level or on a permission level?
        if self.user_matches(rule) and self.rule_applies_to_path(rule):
            for permtype in rule.permissions:
                if not self.terminal[permtype]:
                    self.perms[permtype] = rule.allow
                if rule.terminal:
                    self.terminal[permtype] = True
                    

In [36]:
computed_permission = ComputedPermission.from_user_and_path(cursor, "user@example.org", Path("user@example.org/test2/a.txt"))
computed_permission.has_permission(PermissionType.READ)


True

## Get all read permissions for a set of paths for a user (dir_state)


In [38]:
# Get all table names
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

for table in tables:
    table_name = table[0]
    # Get column info for each table
    columns = cursor.execute(f"PRAGMA table_info({table_name})").fetchall()
    print(f"\nTable: {table_name}")
    print("Columns:")
    for col in columns:
        print(f"  {col[1]} ({col[2]})")



Table: rules
Columns:
  permfile_path (varchar(1000))
  permfile_dir (varchar(1000))
  priority (INTEGER)
  path (varchar(1000))
  user (varchar(1000))
  can_read (bool)
  can_create (bool)
  can_write (bool)
  admin (bool)
  disallow (bool)
  terminal (bool)

Table: file_metadata
Columns:
  id (INTEGER)
  path (TEXT)
  hash (TEXT)
  signature (TEXT)
  file_size (INTEGER)
  last_modified (TEXT)

Table: sqlite_sequence
Columns:
  name ()
  seq ()

Table: rule_files
Columns:
  permfile_path (varchar(1000))
  priority (INTEGER)
  file_id (INTEGER)
  match_for_email (varchar(1000))


In [92]:
def get_read_permissions_for_user(user: str):
    res = cursor.execute("""
    SELECT path,
    (
        SELECT COALESCE(max(
            CASE 
                WHEN can_read AND NOT disallow AND NOT terminal THEN rule_prio
                WHEN can_read AND NOT disallow AND terminal THEN terminal_prio
                ELSE 0
            END
        ) >
        max(
            CASE
                WHEN can_read AND disallow AND NOT terminal THEN rule_prio
                WHEN can_read AND disallow AND terminal THEN terminal_prio
                ELSE 0
            END
        ), 0)  
        FROM (
            SELECT can_read, disallow, terminal,
                row_number() OVER (ORDER BY rules.priority DESC) AS rule_prio,
                row_number() OVER (ORDER BY rules.priority ASC) * 1000000 AS terminal_prio
            FROM rule_files
            JOIN rules ON rule_files.permfile_path = rules.permfile_path and rule_files.priority = rules.priority
            WHERE rule_files.file_id = f.id and (rules.user = ? or rules.user = "*" or rule_files.match_for_email = ?)
        )
    ) AS read_permission
    FROM file_metadata f
    """, (user, user))
    return res.fetchall()


In [105]:
# Clear existing data
cursor.execute("DELETE FROM file_metadata")
cursor.execute("DELETE FROM rule_files") 
cursor.execute("DELETE FROM rules")

# Insert some example file metadata
cursor.execute("""
INSERT INTO file_metadata (id, path, hash, signature, file_size, last_modified) VALUES
    (1, 'user@example.org/test2/a.txt', 'hash1', 'signature1', 100, '2024-01-01'),
    (2, 'user@example.org/test2/b.txt', 'hash2', 'signature2', 200, '2024-01-02'),
    (3, 'user@example.org/test2/c.txt', 'hash3', 'signature3', 300, '2024-01-03')
""")

cursor.execute("""
INSERT INTO rules (permfile_path, permfile_dir, priority, path, user, can_read, can_create, can_write, admin, disallow, terminal) VALUES
    ('user@example.org/test2/.syftperm', 'user@example.org/test2', 1, '*', '*', 1, 0, 0, 0, 0, 0),
    ('user@example.org/test2/.syftperm', 'user@example.org/test2', 2, '*', 'user@example.org', 0, 1, 1, 0, 0, 0),
    ('user@example.org/test2/.syftperm', 'user@example.org/test2', 3, '*', '*', 1, 1, 1, 0, 0, 0)

""")

# Insert example permission rules
cursor.execute("""
INSERT INTO rule_files (permfile_path, priority, file_id, match_for_email) VALUES
    ('user@example.org/test2/.syftperm', 1, 1, NULL),
    ('user@example.org/test2/.syftperm', 2, 2, NULL),
    ('user@example.org/test2/.syftperm', 3, 3, NULL)


""")

connection.commit()

In [106]:
res = [dict(x) for x in get_read_permissions_for_user("user@example.org")]

assert len(res) == 3
assert res[0]["path"] == "user@example.org/test2/a.txt"
assert res[0]["read_permission"] == True
assert res[1]["path"] == "user@example.org/test2/b.txt"
assert res[1]["read_permission"] == False
assert res[2]["path"] == "user@example.org/test2/c.txt"
assert res[2]["read_permission"] == True


# APPENDIX

In [38]:
from wcmatch.glob import globmatch

In [39]:
# r0 c0 w0 a0

# Permission querying prototype

- summary permissions
- types
    - effective permissions
        - read/create/update/update_permissions
        - these are store using bitwise independent permissions bits. Except for update_permissions, we have the admin bit.
            - read/create/update/admin
            - If you are admin, all checks are skipped when checking permissions
            - if you do not have the read bit, you also lose effect write/update
    - roles
        - roles are like aliases. They allow you to give multiple permissions with short syntax 
        - examples
            - admin -> implies admin
            - creator -> implies read+create
            - updater -> implies read+update
            - writer -> implies read+create+update
        - for disallowing, the usage pattern will probably be more fine grained, so you can use permissions like read/create


In [40]:
# x/.syftperm

# r1 1
# r2 2
# r3 3
# rn 4
# r4 5
# r5 6


In [41]:
# Start an in-memory SQLite connection
connection = sqlite3.connect(":memory:")
connection.row_factory = sqlite3.Row  # Set row factory to sqlite3.Row


# Create a cursor object
cursor = connection.cursor()

# Create a table for storing file information
res = cursor.execute("""
    CREATE TABLE rules (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        read bool NOT NULL,
        admin bool NOT NULL,
        disallow bool NOT NULL,
        terminal bool not null
    )
""")

res = cursor.execute("""
    CREATE TABLE paths (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        path varchar(1000) NOT NULL
    )
""")

res = cursor.execute("""
    CREATE TABLE path2rule (
        path_id INTEGER NOT NULL,
        rule_id INTEGER NOT NULL,
        PRIMARY KEY (path_id, rule_id)
    )
""")



In [42]:
# Insert some example file paths
rules = [
    (True, False, False, False),
    (True, False, True, False),
    (False, True, False, False),
    (True, False, False, False),
    (True, False, False, False),
    (True, False, True, False),
    (False, True, False, False),
]

cursor.executemany("INSERT INTO rules (read, admin, disallow, terminal) VALUES (?, ?, ?, ?)", rules)

In [43]:
# Insert some example file paths
paths = [
    ("a@b.openmined.org/a.txt",),
    ("a@b.openmined.org/b.txt",),
    # (True, False, False),
]

cursor.executemany("INSERT INTO paths (path) VALUES (?)", paths)

In [44]:
# Insert some example file paths
path2rule = [
    (1, 1),
    (1, 2),
    (1, 3),
    (1, 4),
    (2, 5),
    (2, 6),
    (2, 7),
]

cursor.executemany("INSERT INTO path2rule (path_id, rule_id) VALUES (?, ?)", path2rule)

In [45]:
def query_all(cursor, table_name):
    res = cursor.execute(f"""
    SELECT * from {table_name}
    """
    )
    for i,x in enumerate(res.fetchall()):
        if i==0:
            print(" | ".join(list(x.keys())))
        print("  |  ".join([str(x) for x in dict(x).values()]))

In [46]:
query_all(cursor, "rules")

id | read | admin | disallow | terminal
1  |  1  |  0  |  0  |  0
2  |  1  |  0  |  1  |  0
3  |  0  |  1  |  0  |  0
4  |  1  |  0  |  0  |  0
5  |  1  |  0  |  0  |  0
6  |  1  |  0  |  1  |  0
7  |  0  |  1  |  0  |  0


In [47]:
query_all(cursor, "paths")

id | path
1  |  a@b.openmined.org/a.txt
2  |  a@b.openmined.org/b.txt


In [48]:
query_all(cursor, "path2rule")

path_id | rule_id
1  |  1
1  |  2
1  |  3
1  |  4
2  |  5
2  |  6
2  |  7


we have

- paths
- rules2paths
- rules


for every path we compute all rules
for all those pathrules we compute all things below

1. we get all paths
2. we make a correlated subquery in the select that 
2a. first gets all the rules
2b. collapses all the rules

In [49]:
# res = cursor.execute("""
# SELECT path, 
# (
#     SELECT max(rule_prio * read * (1-disallow) * (1-terminal) + terminal_prio * read * (1-disallow) * terminal * 1000000) >
#            max(rule_prio * read * disallow * (1-terminal) + terminal_prio * read * disallow * terminal * 1000000) as read_permission
#     from (
#                 SELECT read, disallow, terminal, 
#                 row_number() over (order by id desc) as rule_prio,
#                 row_number() over (order by id asc) as terminal_prio 
#                 FROM path2rule
#                 join rules on path2rule.rule_id == rules.id
#                 where p.id == path2rule.path_id
#     )
# ) as read_permission
# FROM paths p
# """)


In [50]:
read row_number=1 -> 1
read row_number=2 terminal=True -> 90000000
read row_number=3 terminal=True -> 80000000
read row_number=4 -> 4

900000000


SyntaxError: invalid syntax (2559906572.py, line 1)

In [78]:
res = cursor.execute("""
SELECT path, 
(
    SELECT max
    (
        case 
            when read and not disallow and not terminal then rule_prio
            when read and not disallow and terminal then terminal_prio
            else 0
        end
    ) >
    max
    (
        case 
            when read and disallow and not terminal then rule_prio
            when read and disallow and terminal then terminal_prio
            else 0
        end
    )
    from (
                SELECT read, disallow, terminal, 
                row_number() over (order by id desc) as rule_prio,
                row_number() over (order by id asc) * 1000000 as terminal_prio 
                FROM path2rule
                join rules on path2rule.rule_id == rules.id
                where p.id == path2rule.path_id
    )
) as read_permission
FROM paths p
""")


In [79]:
# res = cursor.execute("""
# SELECT path, 
# (
#     SELECT max(allow_score) > max(disallow_score) as read_permission
#     from (
#                 SELECT 
#                 row_number() over (order by id desc) * read * (1-disallow) * (1-terminal) +
#                 row_number() over (order by id asc) * read * (1-disallow) * terminal * 1000000 as allow_score,
#                 row_number() over (order by id desc) * read * disallow * (1-terminal) +
#                 row_number() over (order by id asc) * read * disallow * terminal * 1000000 as disallow_score
#                 FROM path2rule
#                 join rules on path2rule.rule_id == rules.id
#                 where p.id == path2rule.path_id
#     )
# ) as read_permission
# FROM paths p
# """)


In [80]:
for i,x in enumerate(res.fetchall()):
    if i==0:
        print(list(x.keys()))
    print(list(dict(x).values()))

['path', 'read_permission']
['a@b.openmined.org/a.txt', 1]
['a@b.openmined.org/b.txt', 1]


In [63]:
# """
# select max(allow_score) > max(disallow_score) as read_permission
# from (
#             SELECT 
#             row_number() over (order by id desc) * read * (1-disallow) * (1-terminal) +
#             row_number() over (order by id asc) * read * (1-disallow) * terminal * pow(10000, terminal) as allow_score,
#             row_number() over (order by id desc) * read * disallow * (1-terminal) +
#             row_number() over (order by id asc) * read * disallow * terminal * pow(10000, terminal) as disallow_score
#             FROM rules
#             where rules.id in (1,2,3,4)
#             ORDER BY id DESC
#             LIMIT 10
# )
# """

In [64]:
# res = cursor.execute("""
# SELECT path, (
#     select count(*) 
#     from path2rule 
#     join rules on path2rule.rule_id == rules.id
#     where p.id == path2rule.path_id
# )
# FROM paths p
# """)

In [55]:
# res = cursor.execute("""
# SELECT path, path2rule.rule_id as rule_id, rules.read, rules.admin, rules.disallow
# FROM paths
# join path2rule
# on paths.id == path2rule.path_id
# join rules
# on path2rule.rule_id == rules.id
# """)

In [44]:
for i,x in enumerate(res.fetchall()):
    if i==0:
        print(list(x.keys()))
    print(list(dict(x).values()))

['path', 'rule_id', 'read', 'admin', 'disallow']
['a@b.openmined.org/a.txt', 1, 1, 0, 0]
['a@b.openmined.org/a.txt', 1, 1, 0, 1]
['a@b.openmined.org/a.txt', 1, 0, 1, 0]
['a@b.openmined.org/a.txt', 1, 1, 0, 0]
['a@b.openmined.org/a.txt', 1, 1, 0, 0]
['a@b.openmined.org/a.txt', 1, 1, 0, 1]
['a@b.openmined.org/a.txt', 1, 0, 1, 0]
['a@b.openmined.org/a.txt', 2, 1, 0, 0]
['a@b.openmined.org/a.txt', 2, 1, 0, 1]
['a@b.openmined.org/a.txt', 2, 0, 1, 0]
['a@b.openmined.org/a.txt', 2, 1, 0, 0]
['a@b.openmined.org/a.txt', 2, 1, 0, 0]
['a@b.openmined.org/a.txt', 2, 1, 0, 1]
['a@b.openmined.org/a.txt', 2, 0, 1, 0]
['a@b.openmined.org/a.txt', 3, 1, 0, 0]
['a@b.openmined.org/a.txt', 3, 1, 0, 1]
['a@b.openmined.org/a.txt', 3, 0, 1, 0]
['a@b.openmined.org/a.txt', 3, 1, 0, 0]
['a@b.openmined.org/a.txt', 3, 1, 0, 0]
['a@b.openmined.org/a.txt', 3, 1, 0, 1]
['a@b.openmined.org/a.txt', 3, 0, 1, 0]
['a@b.openmined.org/a.txt', 4, 1, 0, 0]
['a@b.openmined.org/a.txt', 4, 1, 0, 1]
['a@b.openmined.org/a.txt', 4, 

# APPENDIX

In [14]:
res = cursor.execute("""
SELECT
    has_read_table.has_read,
    has_admin_table.has_admin
FROM
(SELECT COALESCE
    (
        (
            SELECT not disallow as has_read
            FROM rules
            WHERE read = 1
            ORDER BY id DESC
            LIMIT 1
        ), 
        False
    ) as has_read
) as has_read_table 

CROSS JOIN
(SELECT COALESCE
    (
        (
            SELECT not disallow as has_admin
            FROM rules
            WHERE admin = 1
            ORDER BY id DESC
            LIMIT 1
        ), 
        False
    ) as has_admin
) as has_admin_table 
"""
)

id | read | admin | disallow

1  |  1  |  0  |  0

2  |  1  |  0  |  1

3  |  0  |  1  |  0

4  |  1  |  0  |  0

5  |  1  |  0  |  0

6  |  1  |  0  |  1

7  |  0  |  1  |  0

In [25]:
res = cursor.execute("""
select max(allow_score) > max(disallow_score) as read_permission
from (
            SELECT 
            row_number() over (order by id desc) * read * (1-disallow) * (1-terminal) +
            row_number() over (order by id asc) * read * (1-disallow) * terminal * pow(10000, terminal) as allow_score,
            row_number() over (order by id desc) * read * disallow * (1-terminal) +
            row_number() over (order by id asc) * read * disallow * terminal * pow(10000, terminal) as disallow_score
            FROM rules
            where rules.id in (1,2,3,4)
            ORDER BY id DESC
            LIMIT 10
)
""")

In [26]:
for i,x in enumerate(res.fetchall()):
    if i==0:
        print(list(x.keys()))
    print(list(dict(x).values()))

['read_permission']
[1]


In [ ]:
# res = cursor.execute("""
# WITH has_read_table AS (
# SELECT COALESCE
#     (
#         (
#             SELECT not disallow as has_read
#             FROM rules
#             WHERE read = 1
#             ORDER BY id DESC
#             LIMIT 1
#         ), 
#         False
#     ) as has_read
# ),
# has_admin_table AS (
# SELECT COALESCE
#     (
#         (
#             SELECT not disallow as has_admin
#             FROM rules
#             WHERE admin = 1
#             ORDER BY id DESC
#             LIMIT 1
#         ), 
#         False
#     ) as has_admin
# )
# SELECT
#     has_read_table.has_read,
#     has_admin_table.has_admin
# FROM
#     has_read_table
# CROSS JOIN
#     has_admin_table;
# """
# )